# 时间序列分类模型训练示例

本笔记本展示如何使用ML Pipeline训练时间序列分类模型，包括：
1. 数据加载和预处理
2. 模型配置和创建
3. 模型训练
4. 性能评估
5. 模型保存和转换

## 1. 设置环境

首先，我们需要导入必要的库并设置路径。

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 添加父目录到路径
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

# 导入ML Pipeline模块
from ml_pipeline.models.model import (
    create_model, save_model_with_metadata, get_learning_rate_scheduler
)
from ml_pipeline.data_processing.data_processing import load_dataset
from ml_pipeline.data_processing.config_utils import load_config

# 设置绘图样式
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# 设置随机种子
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

## 2. 加载和准备数据

加载之前处理好的数据，并划分训练集、验证集和测试集。

In [ ]:
# 加载处理好的数据
data_path = 'processed_data.npz'
windows, labels, scaler = load_dataset(data_path)

print(f"数据形状:")
print(f"窗口: {windows.shape}")
print(f"标签: {labels.shape}")

# 查看标签分布
unique, counts = np.unique(labels, return_counts=True)
print("\n标签分布:")
for u, c in zip(unique, counts):
    print(f"  类别 {int(u)} ({['静止', '走路', '跑步'][int(u)]}): {c} 个样本 ({c/len(labels)*100:.1f}%)")

# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(
    windows, labels,
    test_size=0.2,
    random_state=RANDOM_SEED,
    stratify=labels
)

# 从训练集中划分验证集
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train,
    test_size=0.2,
    random_state=RANDOM_SEED,
    stratify=y_train
)

print("\n数据集大小:")
print(f"训练集: {x_train.shape[0]} 个样本")
print(f"验证集: {x_val.shape[0]} 个样本")
print(f"测试集: {x_test.shape[0]} 个样本")

## 3. 配置模型

设置模型架构和训练参数。

In [ ]:
# 加载模型配置
model_config = {
    'model': {
        'architecture': 'lstm',  # 'lstm', 'cnn', 或 'hybrid'
        'layer_sizes': [64, 32],
        'dropout_rate': 0.3,
        'activation': 'tanh',
        'use_batch_norm': True,
        'cnn': {
            'filters': [64, 128],
            'kernel_size': 3,
            'pool_size': 2
        }
    },
    'training': {
        'batch_size': 32,
        'epochs': 50,
        'learning_rate': 0.001,
        'optimizer': 'adam',
        'early_stopping_patience': 10,
        'use_class_weights': True,
        'lr_scheduler': {
            'type': 'reduce_on_plateau',
            'factor': 0.5,
            'patience': 5,
            'min_lr': 0.00001
        }
    }
}

# 打印配置摘要
print("模型配置:")
print("==========")
print(f"架构: {model_config['model']['architecture']}")
print(f"层大小: {model_config['model']['layer_sizes']}")
print(f"Dropout率: {model_config['model']['dropout_rate']}")
print(f"激活函数: {model_config['model']['activation']}")
print(f"使用批归一化: {model_config['model']['use_batch_norm']}")

print("\n训练配置:")
print("==========")
print(f"批大小: {model_config['training']['batch_size']}")
print(f"训练轮数: {model_config['training']['epochs']}")
print(f"学习率: {model_config['training']['learning_rate']}")
print(f"优化器: {model_config['training']['optimizer']}")
print(f"早停耐心值: {model_config['training']['early_stopping_patience']}")

## 4. 创建模型

使用配置创建模型。

In [ ]:
# 获取输入形状和类别数
input_shape = (x_train.shape[1], x_train.shape[2])
num_classes = len(np.unique(y_train))

# 创建模型
model = create_model(
    config=model_config['model'],
    input_shape=input_shape,
    num_classes=num_classes
)

# 显示模型结构
model.summary()

## 5. 准备训练回调函数

设置训练过程中使用的回调函数。

In [ ]:
# 创建回调函数列表
callbacks = []

# 早停回调
if model_config['training']['early_stopping_patience'] > 0:
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=model_config['training']['early_stopping_patience'],
        restore_best_weights=True,
        verbose=1
    )
    callbacks.append(early_stopping)

# 学习率调度器
lr_scheduler = get_learning_rate_scheduler(model_config['training'])
if lr_scheduler:
    callbacks.append(lr_scheduler)

# TensorBoard回调
log_dir = os.path.join('logs', 'fit', pd.Timestamp.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1
)
callbacks.append(tensorboard_callback)

print(f"使用的回调函数:")
for callback in callbacks:
    print(f"  - {callback.__class__.__name__}")

## 6. 计算类别权重

为不平衡的类别分布计算权重。

In [ ]:
# 计算类别权重
if model_config['training']['use_class_weights']:
    unique, counts = np.unique(y_train, return_counts=True)
    total = len(y_train)
    class_weights = {}
    
    for i, count in zip(unique, counts):
        class_weights[int(i)] = total / (len(unique) * count)
    
    print("类别权重:")
    for class_id, weight in class_weights.items():
        print(f"  类别 {class_id} ({['静止', '走路', '跑步'][class_id]}): {weight:.4f}")
else:
    class_weights = None

## 7. 训练模型

使用准备好的数据和配置训练模型。

In [ ]:
# 训练模型
history = model.fit(
    x_train, y_train,
    batch_size=model_config['training']['batch_size'],
    epochs=model_config['training']['epochs'],
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

## 8. 可视化训练过程

绘制训练和验证指标的变化。

In [ ]:
# 绘制训练历史
plt.figure(figsize=(12, 4))

# 损失曲线
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='训练损失')
plt.plot(history.history['val_loss'], label='验证损失')
plt.title('模型损失')
plt.xlabel('轮数')
plt.ylabel('损失')
plt.legend()
plt.grid(True)

# 准确率曲线
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='训练准确率')
plt.plot(history.history['val_accuracy'], label='验证准确率')
plt.title('模型准确率')
plt.xlabel('轮数')
plt.ylabel('准确率')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 9. 评估模型

在测试集上评估模型性能。

In [ ]:
# 在测试集上评估
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"测试集损失: {test_loss:.4f}")
print(f"测试集准确率: {test_acc:.4f}")

# 获取预测结果
y_pred = model.predict(x_test)
if y_pred.shape[1] > 1:
    y_pred = np.argmax(y_pred, axis=1)
else:
    y_pred = (y_pred > 0.5).astype(int).flatten()

# 计算混淆矩阵
cm = tf.math.confusion_matrix(y_test, y_pred)

# 绘制混淆矩阵
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=['静止', '走路', '跑步'],
    yticklabels=['静止', '走路', '跑步']
)
plt.title('混淆矩阵')
plt.xlabel('预测标签')
plt.ylabel('真实标签')
plt.show()

# 计算每个类别的性能指标
for i in range(num_classes):
    class_name = ['静止', '走路', '跑步'][i]
    precision = cm[i, i] / np.sum(cm[:, i])
    recall = cm[i, i] / np.sum(cm[i, :])
    f1 = 2 * (precision * recall) / (precision + recall)
    
    print(f"\n类别 {i} ({class_name}):")
    print(f"  精确率: {precision:.4f}")
    print(f"  召回率: {recall:.4f}")
    print(f"  F1分数: {f1:.4f}")

## 10. 可视化错误预测

查看一些被模型错误分类的样本。

In [ ]:
# 找出错误预测的样本
errors = np.where(y_pred != y_test)[0]
print(f"错误预测数量: {len(errors)}")

# 显示前3个错误预测的样本
n_examples = min(3, len(errors))
plt.figure(figsize=(15, 4*n_examples))

for i, idx in enumerate(errors[:n_examples]):
    # 加速度数据
    plt.subplot(n_examples, 2, i*2+1)
    plt.plot(x_test[idx, :, 0], label='X')
    plt.plot(x_test[idx, :, 1], label='Y')
    plt.plot(x_test[idx, :, 2], label='Z')
    plt.title(f'错误 {i+1} - 加速度\n真实: {["静止", "走路", "跑步"][int(y_test[idx])]} | 预测: {["静止", "走路", "跑步"][int(y_pred[idx])]}')
    plt.xlabel('时间点')
    plt.ylabel('加速度')
    plt.legend()
    plt.grid(True)
    
    # 陀螺仪数据
    plt.subplot(n_examples, 2, i*2+2)
    plt.plot(x_test[idx, :, 3], label='X')
    plt.plot(x_test[idx, :, 4], label='Y')
    plt.plot(x_test[idx, :, 5], label='Z')
    plt.title(f'错误 {i+1} - 陀螺仪\n真实: {["静止", "走路", "跑步"][int(y_test[idx])]} | 预测: {["静止", "走路", "跑步"][int(y_pred[idx])]}')
    plt.xlabel('时间点')
    plt.ylabel('角速度')
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()

## 11. 保存模型

保存训练好的模型和相关元数据。

In [ ]:
# 创建输出目录
output_dir = 'trained_models'
os.makedirs(output_dir, exist_ok=True)

# 保存模型
model_name = f"{model_config['model']['architecture']}_model"
model_path = save_model_with_metadata(
    model=model,
    output_dir=output_dir,
    model_name=model_name,
    config=model_config,
    input_shape=input_shape,
    num_classes=num_classes,
    class_names=['静止', '走路', '跑步'],
    feature_names=['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']
)

print(f"模型已保存到 {model_path}")

## 12. 使用命令行工具

展示如何使用命令行工具进行模型训练和评估。

In [ ]:
print("训练命令示例:")
print("train-model \
    --data processed_data.npz \
    --config model_config.yaml \
    --output-dir trained_models \
    --name lstm_model \
    --epochs 50 \
    --batch-size 32 \
    --learning-rate 0.001")

print("\n评估命令示例:")
print("evaluate-model \
    --model trained_models/lstm_model.h5 \
    --data test_data.npz \
    --output-dir evaluation \
    --class-names 静止,走路,跑步")

## 13. 总结

在本笔记本中，我们展示了如何使用ML Pipeline进行时间序列分类模型的训练：

1. 加载和准备数据
2. 配置和创建模型
3. 设置训练参数和回调函数
4. 训练模型
5. 评估模型性能
6. 可视化结果
7. 保存模型

这个流程可以作为开发自己的时间序列分类模型的起点。通过调整模型架构、超参数和训练策略，你可以进一步提高模型性能。